In [2]:
import os
import sys

# Add the path to the directory containing your project to the Python path
project_path = '/home/ted/Desktop/DEV/telecom-user-behavior-analysis'
sys.path.append(project_path)

import pandas as pd
from DB_connection.connection import PostgresConnection
from src.utils import missing_values_table, convert_bytes_to_megabytes

# Change the current working directory to the project directory
os.chdir(project_path)

Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

In [3]:
# Connect to the PostgreSQL database
db = PostgresConnection(dbname='postgres_db', user='postgres_username', password='postgres_password')
db.connect()

# Execute query to retrieve data
query = "SELECT * FROM xdr_data"
result = db.execute_query(query)

# Convert result to Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
print(df.head())  # Display the first few rows of the DataFrame

# Close the database connection
db.close_connection()

Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

In [4]:
df['HTTP DL (MB)'] = convert_bytes_to_megabytes(df, 'HTTP DL (Bytes)')

print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube UL (Bytes)  Netflix DL (Bytes)  \
0  9.16456699548519E+015  ...           2501332.0           8198936.0   
1                L77566A  ...          19111729.0          18338413.0   

In [5]:
#Task 3.1 
#1. Average TCP Retransmission
average_tcp_retransmission = df.groupby('MSISDN/Number')[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']].mean().sum(axis=1)

# 2. Average RTT
average_rtt = df.groupby('MSISDN/Number')[['Avg RTT DL (ms)', 'Avg RTT UL (ms)']].mean().sum(axis=1)

# 3. Handset Type
most_frequent_handset_type = df.groupby('MSISDN/Number')['Handset Type'].agg(lambda x: x.value_counts().index[0])

# 4. Average Throughput
average_throughput = df.groupby('MSISDN/Number')[['Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']].mean().sum(axis=1)


# Aggregate all information into a DataFrame
customer_engagement_metrics = pd.DataFrame({
    'MSISDN/Number': average_tcp_retransmission.index,
    'Average TCP Retransmission': average_tcp_retransmission.values,
    'Average RTT': average_rtt.values,
    'Handset Type': most_frequent_handset_type.values,
    'Average Throughput': average_throughput.values
})

# Display the aggregated information
print(customer_engagement_metrics.head())


   MSISDN/Number  Average TCP Retransmission  Average RTT  \
0   3.360100e+10                         0.0         46.0   
1   3.360100e+10                         0.0         31.0   
2   3.360100e+10                         0.0          0.0   
3   3.360101e+10                      1066.0         84.0   
4   3.360101e+10                   9370832.0         59.5   

                     Handset Type  Average Throughput  
0  Huawei P20 Lite Huawei Nova 3E                76.0  
1          Apple iPhone 7 (A1778)                99.0  
2                       undefined                97.0  
3         Apple iPhone 5S (A1457)               248.0  
4         Apple iPhone Se (A1723)             28422.0  


In [7]:
#Task 3.2
# Top 10 TCP values
top_10_tcp_values = df['TCP DL Retrans. Vol (Bytes)'].value_counts().head(10)
print("Top 10 TCP values:")
print(top_10_tcp_values)

# Bottom 10 TCP values
bottom_10_tcp_values = df['TCP DL Retrans. Vol (Bytes)'].value_counts().tail(10)
print("\nBottom 10 TCP values:")
print(bottom_10_tcp_values)

# Most frequent TCP value
most_frequent_tcp_value = df['TCP DL Retrans. Vol (Bytes)'].mode().iloc[0]
print("\nMost frequent TCP value:", most_frequent_tcp_value)

# Top 10 RTT values
top_10_rtt_values = df['Avg RTT DL (ms)'].value_counts().head(10)
print("\nTop 10 RTT values:")
print(top_10_rtt_values)

# Bottom 10 RTT values
bottom_10_rtt_values = df['Avg RTT DL (ms)'].value_counts().tail(10)
print("\nBottom 10 RTT values:")
print(bottom_10_rtt_values)

# Most frequent RTT value
most_frequent_rtt_value = df['Avg RTT DL (ms)'].mode().iloc[0]
print("\nMost frequent RTT value:", most_frequent_rtt_value)

# Top 10 Throughput values
top_10_throughput_values = df['Avg Bearer TP DL (kbps)'].value_counts().head(10)
print("\nTop 10 Throughput values:")
print(top_10_throughput_values)

# Bottom 10 Throughput values
bottom_10_throughput_values = df['Avg Bearer TP DL (kbps)'].value_counts().tail(10)
print("\nBottom 10 Throughput values:")
print(bottom_10_throughput_values)

# Most frequent Throughput value
most_frequent_throughput_value = df['Avg Bearer TP DL (kbps)'].mode().iloc[0]
print("\nMost frequent Throughput value:", most_frequent_throughput_value)


Top 10 TCP values:
TCP DL Retrans. Vol (Bytes)
1330.0    433
2660.0    219
38.0      200
92.0      192
1318.0    156
543.0     133
3990.0    114
76.0      108
5320.0    102
1288.0     93
Name: count, dtype: int64

Bottom 10 TCP values:
TCP DL Retrans. Vol (Bytes)
67192.0       1
3141346.0     1
6239605.0     1
502066.0      1
1541097.0     1
12686.0       1
18256688.0    1
3550956.0     1
18957898.0    1
16552848.0    1
Name: count, dtype: int64

Most frequent TCP value: 1330.0

Top 10 RTT values:
Avg RTT DL (ms)
28.0    3553
27.0    3380
34.0    3296
35.0    2892
37.0    2718
36.0    2645
38.0    2547
29.0    2498
26.0    2488
33.0    2430
Name: count, dtype: int64

Bottom 10 RTT values:
Avg RTT DL (ms)
1130.0    1
6173.0    1
5672.0    1
2929.0    1
1243.0    1
2771.0    1
3312.0    1
1413.0    1
6141.0    1
4274.0    1
Name: count, dtype: int64

Most frequent RTT value: 28.0

Top 10 Throughput values:
Avg Bearer TP DL (kbps)
23.0    7324
43.0    4352
44.0    4186
6.0     3983
22.0  

In [9]:
#Task 3.3
# Distribution of average throughput per handset type
average_throughput_per_handset = df.groupby('Handset Type')['Avg Bearer TP DL (kbps)'].mean()
print("Distribution of average throughput per handset type:")
print(average_throughput_per_handset)

# Average TCP retransmission per handset type
average_tcp_retransmission_per_handset = df.groupby('Handset Type')[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']].mean().sum(axis=1)
print("\nAverage TCP retransmission per handset type:")
print(average_tcp_retransmission_per_handset)



Distribution of average throughput per handset type:
Handset Type
A-Link Telecom I. Cubot A5                                                             23184.000000
A-Link Telecom I. Cubot Note Plus                                                       6527.000000
A-Link Telecom I. Cubot Note S                                                          8515.000000
A-Link Telecom I. Cubot Nova                                                           55667.000000
A-Link Telecom I. Cubot Power                                                          69126.000000
                                                                                           ...     
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809       44.500000
Zyxel Communicat. Lte7460                                                              39741.000000
Zyxel Communicat. Sbg3600                                                              82818.000000
Zyxel Communicat. Zyxel Wah7706   

In [14]:
#Task 3.4
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Selecting relevant features
features = customer_engagement_metrics[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']]

# Scaling the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
customer_engagement_metrics['Cluster'] = kmeans.fit_predict(scaled_features)
print(customer_engagement_metrics['Cluster'])


0         1
1         1
2         1
3         1
4         1
         ..
106851    1
106852    1
106853    1
106854    1
106855    1
Name: Cluster, Length: 106856, dtype: int32
